# Augment Entity DataFrame with Translation Columns

## Is using paid translation service from Googld Cloud Translate
Put the path of google application credential here

In [9]:
%env GOOGLE_APPLICATION_CREDENTIALS=../GOOGLE_APPLICATION_CREDENTIALS/AIDA-0cf5a4cb6e90.json

env: GOOGLE_APPLICATION_CREDENTIALS=../GOOGLE_APPLICATION_CREDENTIALS/AIDA-0cf5a4cb6e90.json


In [10]:
import csv
import pandas as pd
import html
import sys
from ast import literal_eval as make_tuple
from langdetect import detect

## Original Entity Dataframe

### Params

In [11]:
repo = 'dryrun3-gaia2v1-vision-ta2'
version = '001'
store_data_dir = '/nas/home/jchen/store_data/' + repo

In [12]:
df_entity = pd.read_hdf(store_data_dir + '/entity_all_' + version + '.h5')
df_entity = df_entity.where(pd.notnull(df_entity), None)
has_origin = True if 'originLabel' in df_entity.columns else False
df_entity.head()

,e,type,name,source,targets,target_scores,fbid,fbid_score_avg,fbid_score_max,wikidata,wiki_label_en,wiki_label_ru,wiki_label_uk,wiki_alias_en,wiki_alias_ru,wiki_alias_uk,lang,label
0,http://www.isi.edu/gaia/entities/3a91d6b4-1e0d...,ldcOnt:GPE,"(Гвинее,)",HC00007OW,None,None,None,None,None,None,None,None,None,None,None,None,ru,None
4,http://www.isi.edu/gaia/entities/15dc8f2c-b83e...,ldcOnt:GPE.Country.Country,"(Гамбии, Гамбии, Гамбии)",HC00007OW,"(LDC2019E43:2409223, LDC2019E43:2413451, LDC20...","(0.0014, 0.9935, 0.0014)","(m.0hdx8,)","(0.6491159797,)","(0.6491159797,)","(http://www.wikidata.org/entity/Q1005,)","(Gambia,)","(Гамбия,)","(Гамбія,)","((gm, GAM, 🇬🇲, Islamic Republic of the Gambia,...","((Республика Гамбия,),)","((Республіка Гамбія,),)",ru,"((Gambia ,),)"
7,http://www.isi.edu/gaia/entities/4bd2e87f-78b3...,ldcOnt:GPE.Country.Country,"(Италию, Италию, Италию, Италию, Италию, Итали...",HC00007OW,"(LDC2019E43:3170325, LDC2019E43:3175395, LDC20...","(0.0001456717, 0.9994537, 0.0001456717, 0.0002...","(m.03rjj,)","(0.8315719711,)","(1.0,)","(http://www.wikidata.org/entity/Q38,)","(Italy,)","(Италия,)","(Італія,)","((ITA, it, IT, 🇮🇹, Italia, Italian Republic),)","((Итальянская Республика,),)","((),)",ru,"((Italy ,),)"
17,http://www.isi.edu/gaia/entities/08174a18-570b...,ldcOnt:GPE,"(провинции Турина,)",HC00007OW,None,None,None,None,None,None,None,None,None,None,None,None,ru,None
18,http://www.isi.edu/gaia/entities/16c350bb-352d...,ldcOnt:VAL,None,HC00007OW,None,None,None,None,None,None,None,None,None,None,None,None,ru,None


## Get the dictionaries
Load previously generated Russian and Ukranian dictionaries

In [ ]:
def get_dictionary(file_path):
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        return {row['RU/UK'].lower():row['EN'] for row in reader}

In [ ]:
dict_ru_path = 'dictionaries/dict_ru_en.csv'
dict_uk_path = 'dictionaries/dict_uk_en.csv'
dict_ru = get_dictionary(dict_ru_path)
dict_uk = get_dictionary(dict_uk_path)

## Translation Function
Translate a list of words. Dictionaries are used before using Google translate API (because it's fee-based). Save new translations in the dictionary.

In [ ]:
def is_en(text):
    try:
        if detect(text) != 'ru' and detect(text) != 'uk':
            return True
    except:
        return True
    return False

def translate_words(word_list, source_lang):
    dict_a = None
    if source_lang.lower() == 'ru':
        dict_a = dict_ru
    else:
        dict_a = dict_uk
        
    translations = []
    for word in word_list:
        translation = None
        if is_en(word):
            translations.append(word)
        elif word.lower() in dict_a:
            translations.append(dict_a[word.lower()])
        else: # else use google translate and add to dictionary
            print('\r', word, end='')
            translated = free_google_translate(word, source_lang)
            translations.append(translated)
            dict_a[word.lower()] = translated # write out the new dictionary??
    return translations

# free google translator
from googletrans import Translator
free_translator = Translator()
def free_google_translate(text, source_lang):
    try:
        translation = free_translator.translate(text, src=source_lang, dest='en')
    except:
        return None
    return translation.text
    
def free_google_translate_bulk(words, source_lang):
    requests = list(partition_list(words, 100))
    res = {}
    for request in requests:
        try:
            translations = free_translator.translate(request, src=source_lang, dest='en')
            for translation in translations:
                res[translation.origin] = translation.text
        except:
            pass
    return res

# paid google cloud translator
from google.cloud import translate
google_translate_client = translate.Client()

def google_translate(text, source_lang):
    translation = google_translate_client.translate(text, source_language=source_lang, target_language='EN')
    return html.unescape(translation['translatedText'])

def google_translate_bulk(words, source_lang):
    requests = list(partition_list(words, 100))
    dict_a = {}
    for request in requests:
        translations = google_translate_client.translate(request, source_language=source_lang, target_language='EN')
        for translation in translations:
            dict_a[translation['input'].lower()] = html.unescape(translation['translatedText'])
    return dict_a

def partition_list(lines: list, size: int):
    for i in range(0, len(lines), size):
        yield lines[i:i+size]

## Get a list of strings from dataframe not in the dictionary
Avoiding translating as we go through the dataframe, it's very slow

In [ ]:
def get_all_ru_uk(df):
    lists = {}
    ru = []
    uk = []
    for i, row in df.iterrows():
        if row['name'] and row.lang:
            if row.lang.lower()[:2] == 'ru':
                ru = ru + list(row['name'])
            elif row.lang.lower()[:2] == 'uk':
                uk = uk + list(row['name'])
        if row.wiki_label_ru:
            ru = ru + list(row.wiki_label_ru)
        if row.wiki_label_uk:
            uk = uk + list(row.wiki_label_uk)
        if row.wiki_alias_ru:
            for lst in row.wiki_alias_ru:
                if lst:
                    ru = ru + list(lst)
        if row.wiki_alias_uk:
            for lst in row.wiki_alias_uk:
                if lst:
                    uk = uk + list(lst)
        if has_origin and row.originLabel and row.lang:
            if row.lang.lower()[:2] == 'ru':
                ru = ru + list(row.originLabel)
            elif row.lang.lower()[:2] == 'uk':
                uk = uk + list(row.originLabel)
    ru = list(set(ru)) # remove duplicates
    uk = list(set(uk))
    ru = list(w for w in ru if not is_en(w) and w.lower() not in (word.lower() for word in dict_ru))
    uk = list(w for w in uk if not is_en(w) and w.lower() not in (word.lower() for word in dict_uk))
    
    lists['ru'] = list(set(ru))
    lists['uk'] = list(set(uk))

    return lists
        
need_transl = get_all_ru_uk(df_entity)

### Translate those not in the dictionary in bulk

In [ ]:
dict_ru_ext = free_google_translate_bulk(need_transl['ru'], 'RU')
dict_ru.update(dict_ru_ext)
dict_uk_ext = free_google_translate_bulk(need_transl['uk'], 'UK')
dict_uk.update(dict_uk_ext)

## Save dictionaries

In [ ]:
def save_dictionary(dict_a, file):       
    with open(file, 'w') as csvfile:
        fieldnames = ['RU/UK', 'EN']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for k, v in dict_a.items():
            writer.writerow({'RU/UK': k, 'EN': v})

save_dictionary(dict_ru, dict_ru_path)
save_dictionary(dict_uk, dict_uk_path)

## The core: Adding translation columns to entity table

In [ ]:
def add_translation_cols(table):
    if has_origin:
        table[['transl_name', 'transl_label_ru', 'transl_label_uk', 'transl_alias_ru', 'transl_alias_uk', 'transl_origin_label']] = table[['name', 'lang', 'wiki_label_ru', 'wiki_label_uk', 'wiki_alias_ru', 'wiki_alias_uk', 'originLabel']].apply(get_translation_cols, axis='columns')
    else:
        table[['transl_name', 'transl_label_ru', 'transl_label_uk', 'transl_alias_ru', 'transl_alias_uk']] = table[['name', 'lang', 'wiki_label_ru', 'wiki_label_uk', 'wiki_alias_ru', 'wiki_alias_uk']].apply(get_translation_cols, axis='columns')
    return table
   
def get_translation_cols(row):
    transl_label = None if not row['name'] or not row.lang or row.lang.lower().startswith('en') else translate_words(list(row['name']), row.lang[:2])
    transl_label_ru = translate_words(list(row.wiki_label_ru), 'RU') if row.wiki_label_ru else None 
    transl_label_uk = translate_words(list(row.wiki_label_uk), 'UK') if row.wiki_label_uk else None
    transl_alias_ru = ()
    if row.wiki_alias_ru:
        for lst in row.wiki_alias_ru:
            if lst:
                transl_alias_ru = transl_alias_ru + (translate_words(list(lst), 'RU'),)
            else:
                transl_alias_ru = transl_alias_ru + (None,)
    else:
        transl_alias_ru = None
    
    transl_alias_uk = ()
    if row.wiki_alias_uk:
        for lst in row.wiki_alias_uk:
            if lst:
                transl_alias_uk = transl_alias_uk + (translate_words(list(lst), 'UK'),)
            else:
                transl_alias_uk = transl_alias_uk + (None,)
    else:
        transl_alias_uk = None
    
    if has_origin:
        transl_origin_label = None if not row.originLabel or not row.lang or row.lang.lower().startswith('en') else translate_words(list(row.originLabel), row.lang[:2])
        return pd.Series({'transl_name': transl_label, 'transl_label_ru': transl_label_ru, 'transl_label_uk': transl_label_uk, 'transl_alias_ru': transl_alias_ru, 'transl_alias_uk': transl_alias_uk, 'transl_origin_label': transl_origin_label})
    else:
        return pd.Series({'transl_name': transl_label, 'transl_label_ru': transl_label_ru, 'transl_label_uk': transl_label_uk, 'transl_alias_ru': transl_alias_ru, 'transl_alias_uk': transl_alias_uk})

df_trans = add_translation_cols(df_entity)


 царской империи

## Write out new entity dataframe

In [ ]:
# write out dataframe
df_trans.to_hdf(store_data_dir + '/entity_trans_all_' + version + '.h5', 'entity', mode='w', format='fixed')
_ = pd.read_hdf(store_data_dir + '/entity_trans_all_' + version + '.h5')

# write out dataframe filtered
# df_trans_filtered = df_trans[(~df_trans['debug'])]
df_trans_filtered = df_trans
df_trans_filtered.to_hdf(store_data_dir + '/entity_trans_all_filtered_' + version + '.h5', 'entity', mode='w', format='fixed')
df_trans_filtered.to_csv(store_data_dir + '/entity_trans_all_filtered_' + version + '.csv')

In [ ]:
df_trans = pd.read_hdf(store_data_dir + '/entity_trans_all_filtered_' + version + '.h5')
df_trans.head()

## Save dictionary

In [ ]:
save_dictionary(dict_ru, dict_ru_path)
save_dictionary(dict_uk, dict_uk_path)